In [16]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

! pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Association Rule for Store Dataset

In this case study, we will explore how association rule can be used to analyze the items that are usualy purcased together.

## Load Data

We will use the dataset of the transaction in a certain store. You can get the dataset here: 
https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv

In [2]:
# load the data set ans show the first five transaction
url = "https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv"
df = pd.read_csv(url)
df.head(5)

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


# Get the set of product that has been purchased


In [3]:
barang = set()
for col in df:
    barang.update(df[col].unique())
print(barang)

{nan, 'Meat', 'Bread', 'Cheese', 'Pencil', 'Diaper', 'Wine', 'Milk', 'Eggs', 'Bagel'}


## Preprocess Data

In this step, we will transform our dataset so that we will have a one hot encoding based on the purchased products.

In [4]:
#create an itemset based on the products
# encoding the feature
setbarang = set(barang)
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(setbarang - rowset)
    commons = list(setbarang.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]

{nan: 0,
 'Bagel': 0,
 'Milk': 0,
 'Meat': 1,
 'Bread': 1,
 'Cheese': 1,
 'Pencil': 1,
 'Diaper': 1,
 'Wine': 1,
 'Eggs': 1}

In [5]:
  # create new dataframe from the encoded features
df_baru = pd.DataFrame(encoded_vals)
  # show the new dataframe
df_baru.head(5)

,NaN,Bagel,Milk,Meat,Bread,Cheese,Pencil,Diaper,Wine,Eggs
0,0,0,0,1,1,1,1,1,1,1
1,0,0,1,1,1,1,1,1,1,0
2,1,0,1,1,0,1,0,0,1,1
3,1,0,1,1,0,1,0,0,1,1
4,1,0,0,1,0,0,1,0,1,0


Since, the encoded dataframe consist of the empty column. We will drop the NaN column or select all columns other than the first column.

In [6]:
df_baru.drop(columns=df_baru.columns[0], axis=1, inplace=True)

In [7]:
df_baru.head(5)

,Bagel,Milk,Meat,Bread,Cheese,Pencil,Diaper,Wine,Eggs
0,0,0,1,1,1,1,1,1,1
1,0,1,1,1,1,1,1,1,0
2,0,1,1,0,1,0,0,1,1
3,0,1,1,0,1,0,0,1,1
4,0,0,1,0,0,1,0,1,0


## Apriori Algorithm

We will use appriori algorithm to determine the frequently purchased products. 
For this case study, we will min_support=0.2

In [20]:
freq_barang = apriori(df_baru, min_support=0.2, use_colnames=True)
freq_barang

,support,itemsets
0,0.425397,(Bagel)
1,0.501587,(Milk)
2,0.476190,(Meat)
3,0.504762,(Bread)
4,0.501587,(Cheese)
5,0.361905,(Pencil)
6,0.406349,(Diaper)
7,0.438095,(Wine)
8,0.438095,(Eggs)
9,0.225397,"(Bagel, Milk)"


The we will generate association rule of the frequent itemset based on confidence level with the threshold=0.6

In [21]:
basketrule = association_rules(freq_barang, metric="confidence", min_threshold=0.6)
basketrule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Cheese),(Milk),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
2,(Milk),(Cheese),0.501587,0.501587,0.304762,0.607595,1.211344,0.053172,1.270148
3,(Meat),(Cheese),0.476190,0.501587,0.323810,0.680000,1.355696,0.084958,1.557540
4,(Cheese),(Meat),0.501587,0.476190,0.323810,0.645570,1.355696,0.084958,1.477891
5,(Eggs),(Meat),0.438095,0.476190,0.266667,0.608696,1.278261,0.058050,1.338624
6,(Wine),(Cheese),0.438095,0.501587,0.269841,0.615942,1.227986,0.050098,1.297754
7,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
8,"(Meat, Cheese)",(Milk),0.323810,0.501587,0.203175,0.627451,1.250931,0.040756,1.337845
9,"(Meat, Milk)",(Cheese),0.244444,0.501587,0.203175,0.831169,1.657077,0.080564,2.952137
